In [10]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25 python-dotenv langchain_chroma langchain


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
%load_ext dotenv
%dotenv

In [6]:
import warnings

warnings.filterwarnings("ignore")

In [11]:
from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings
from langchain.docstore.document import Document

sample_text_list = [
  "마법사의 나무는 마법 지팡이의 재료 중 최상급의 재료로 알려져 있다",
  "There are Weasleys' Wizard Wheezes nearby the Diagon Alley",
  "Diangon Alley is a place where you can buy magic wands",
  "Hogwart is a school for witches and wizards",
]
sample_docs = [Document(page_content=text) for text in sample_text_list]
vectorstore = Chroma.from_documents(
    documents=sample_docs,
    embedding=UpstageEmbeddings(model="solar-embedding-1-large"),
)
retriever = vectorstore.as_retriever()

In [20]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    You are best D&D host. And user is playing D&D game with you.
    Please answer in a exciting tone as a host.
    User has to make choices in the game.
    User has 10 life points and 10 gold coins.
    Please make scenario of most interesting event from the following context, and provide diverse choices which user can select.
    For each round, you should provide at least 3 choices and total round should be less than 50.
    You should explain situation and choices in detail.
    If someone's conversation is included in the scenario, please express it in colloquial terms that fit the person's character as much as possible.
    Choices should be diverse and interesting, and each choice will reduce or increase user's life points or gold coins.
    If user make appropriate decision, user's life points or gold coins will increase.
    If user make inappropriate decision, user's life points or gold coins will decrease and decreasing amount should not be larger than amount of user owned.
    When user select a choice, you should provide the result of the choice, but you should not show result before user select the choice.
    You should consider previous conversation and context to make the scenario.
    If user's life points or gold coins are less than 0, user will lose the game and get bad ending.
    Also when total round is more than 50, user will lose the game and get bad ending.
    If user's life points or gold coins are more than 20, user will win the game and get good ending.
    ---
    User Choice: {user_choice}
    ---
    Previous Conversation: {previous_conversation}
    ---
    Context: {context}
    """
)
chain = prompt_template | llm | StrOutputParser()



In [21]:
user_choice = "저는 마법사의 나무와 용의 심장으로 만든 지팡이를 사고 싶어요."
sample_context = "What is related information about {user_choice}?"
result_docs = retriever.invoke(sample_context)
chain.invoke({ "user_choice": user_choice, "previous_conversation": "", "context": sample_context })

'마법사의 나무와 용의 심장으로 만든 지팡이를 사고 싶다는군요! 그것은 강력한 마법 아이템입니다. 이제 선택의 시간이 왔습니다. 이 지팡이를 어떻게 얻을 것인가요?\n\n1. **마법 상점을 방문하세요:** 마을의 마법 상점을 찾아가서 지팡이를 구매해보세요. 가격은 8골드 코인입니다.\n2. **마법사로부터 구매하세요:** 마을에 있는 마법사에게 지팡이를 구매할 수 있는지 물어보세요. 하지만 가격은 10골드 코인으로 비싸집니다.\n3. **지팡이를 제작하세요:** 재료와 마법 지식을 가지고 지팡이를 직접 제작해보세요. 하지만 이 선택은 5골드 코인을 소비하고, 제작에 성공할 확률은 50%입니다.\n\n어떤 선택을 하시겠습니까?'